In [1]:
from tinc import *

In [2]:
data_dir = r'C:\Users\Andres\source\repos\vdv_data\nbO_2chempot'
config_file = 'mc_settings.json'
parameter_start_key = 'driver/initial_conditions/*'
parameter_end_key = 'driver/final_conditions/*'
parameter_increment_key = 'driver/incremental_conditions/*'

extract_parameter_space_data(data_dir, config_file, parameter_start_key, parameter_end_key, parameter_increment_key)

{'param_chem_pot(a)': {'space': array([3.8, 3.9])},
 'param_chem_pot(b)': {'space': array([-19.9, -19.8])},
 'temperature': {'space': array([  20.,   30.,   40.,   50.,   60.,   70.,   80.,   90.,  100.,
          110.,  120.,  130.,  140.,  150.,  160.,  170.,  180.,  190.,
          200.,  210.,  220.,  230.,  240.,  250.,  260.,  270.,  280.,
          290.,  300.,  310.,  320.,  330.,  340.,  350.,  360.,  370.,
          380.,  390.,  400.,  410.,  420.,  430.,  440.,  450.,  460.,
          470.,  480.,  490.,  500.,  510.,  520.,  530.,  540.,  550.,
          560.,  570.,  580.,  590.,  600.,  610.,  620.,  630.,  640.,
          650.,  660.,  670.,  680.,  690.,  700.,  710.,  720.,  730.,
          740.,  750.,  760.,  770.,  780.,  790.,  800.,  810.,  820.,
          830.,  840.,  850.,  860.,  870.,  880.,  890.,  900.,  910.,
          920.,  930.,  940.,  950.,  960.,  970.,  980.,  990., 1000.,
         1010., 1020., 1030., 1040., 1050., 1060., 1070., 1080., 1090.,
    

In [3]:
data_dir = r'C:\Users\Andres\source\repos\vdv_data\nbO_2chempot'
config_file = 'mc_settings.json'
parameter_start_key = 'driver/initial_conditions/*'
parameter_end_key = 'driver/final_conditions/*'
parameter_increment_key = 'driver/incremental_conditions/*'

ps = make_parameter_space(data_dir, config_file, parameter_start_key, parameter_end_key, parameter_increment_key, ps_name="casmParams")

ps.set_current_path_template("A_%%param_chem_pot(a)%%B_%%param_chem_pot(b)%%")
ps.print()

 ** ParameterSpace casmParams: <tinc.parameter_space.ParameterSpace object at 0x000001EC927BD310>
   -- Parameter param_chem_pot(a) /param_chem_pot(a)
   -- Parameter param_chem_pot(b) /param_chem_pot(b)
   -- Parameter temperature /temperature


In [4]:
dp = DataPoolJson("results", ps, "slice_dir")
dp.register_data_file("results.json", "temperature")

In [5]:
dp.get_current_files()

['C:\\Users\\Andres\\source\\repos\\vdv_data\\nbO_2chempot\\A_3.8B_-19.8\\results.json']

In [6]:
ps.get_current_relative_path()

'A_3.8B_-19.8'

In [7]:
dp.list_fields()

['<atom_frac(Nb)>',
 '<atom_frac(O)>',
 '<comp(a)>',
 '<comp(b)>',
 '<comp_n(Nb)>',
 '<comp_n(O)>',
 '<comp_n(Va)>',
 '<formation_energy>',
 '<potential_energy>',
 '<site_frac(Nb)>',
 '<site_frac(O)>',
 '<site_frac(Va)>',
 'Beta',
 'N_avg_samples',
 'N_equil_samples',
 'T',
 'heat_capacity',
 'is_converged',
 'is_equilibrated',
 'param_chem_pot(a)',
 'param_chem_pot(b)',
 'prec(<atom_frac(Nb)>)',
 'prec(<atom_frac(O)>)',
 'prec(<comp(a)>)',
 'prec(<comp(b)>)',
 'prec(<comp_n(Nb)>)',
 'prec(<comp_n(O)>)',
 'prec(<comp_n(Va)>)',
 'prec(<formation_energy>)',
 'prec(<potential_energy>)',
 'prec(<site_frac(Nb)>)',
 'prec(<site_frac(O)>)',
 'prec(<site_frac(Va)>)',
 'susc_n(Nb,Nb)',
 'susc_n(Nb,O)',
 'susc_n(Nb,Va)',
 'susc_n(O,O)',
 'susc_n(S,Nb)',
 'susc_n(S,O)',
 'susc_n(S,Va)',
 'susc_n(Va,O)',
 'susc_n(Va,Va)',
 'susc_x(S,a)',
 'susc_x(S,b)',
 'susc_x(a,a)',
 'susc_x(a,b)',
 'susc_x(b,b)']

In [8]:
ps.get_dimension("param_chem_pot(a)").value = 3.9

In [9]:
#dp.debug = True
dp.get_slice("<formation_energy>", "temperature")

masked_array(data=[-20.115568, -20.115568, -20.115568, -20.115568,
                   -20.115568, -20.115568, -20.115568, -20.115568,
                   -20.115568, -20.115568, -20.115568, -20.115568,
                   -20.115568, -20.115568, -20.115568, -20.115568,
                   -20.115568, -20.115568, -20.115568, -20.115568,
                   -20.115568, -20.115568, -20.115568, -20.115568,
                   -20.115568, -20.115568, -20.115568, -20.115568,
                   -20.115568, -20.115568, -20.115568, -20.115568,
                   -20.115568, -20.115568, -20.115568, -20.115568,
                   -20.115568, -20.115568, -20.115568, -20.115568,
                   -20.115568, -20.115568, -20.115576, -20.11559 ,
                   -20.115597, -20.115618, -20.11561 , -20.115568,
                   -20.115623, -20.115692, -20.115662, -20.1156  ,
                   -20.115719, -20.11563 , -20.115742, -20.11576 ,
                   -20.115826, -20.11572 , -20.115856, -20.115

In [10]:
len(dp.get_slice("<formation_energy>", "temperature")), len(ps.get_parameter('temperature').values)

(279, 279)

In [11]:
dp.get_slice("<formation_energy>", "param_chem_pot(a)")

masked_array(data=[-20.115568, -20.115568],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [12]:
len(dp.get_slice("<formation_energy>", "param_chem_pot(a)")), len(ps.get_parameter('param_chem_pot(a)').values)

(2, 2)

In [13]:
import netCDF4

ps_filename = "parameter_space.nc"
sub_dir = sub_dirs[0]
full_path = data_dir + subdir + ps_filename
ps_file = netCDF4.Dataset(full_path, "w", format="NETCDF4")

params = ps_file.createGroup("internal_parameters")
mapped_params = ps_file.createGroup("mapped_parameters")
index_params = ps_file.createGroup("index_parameters")

for param_name, space in param_space.items():
    param_group = rootgrp.createVariable("values","f8",("internal_parameters",))
    mapped_group = rootgrp.createVariable("values","f8",("mapped_parameters",))
    mapped_var_ids = rootgrp.createVariable("ids","s",("mapped_parameters",))
    index_group = rootgrp.createVariable("values","f8",("index_parameters",))


param_var = rootgrp.createVariable("values","f8",("internal_parameters",))
mapped_var = rootgrp.createVariable("values","f8",("mapped_parameters",))
mapped_var_ids = rootgrp.createVariable("ids","s",("mapped_parameters",))
index_params = rootgrp.createVariable("values","f8",("index_parameters",))

ps_file.close()

NameError: name 'sub_dirs' is not defined